In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
device = "cuda"

In [2]:
class config:
    model = "resnet18"
    model2 = "resnet18"
    batchsize = 32
    WEIGHT_DECAY = 0.01
    EPOCHS = 20
class path:
    maindir = "/kaggle/input/gsoclensing/dataset/train"
    valid = "/kaggle/input/gsoclensing/dataset/val"

In [3]:
pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
from tqdm import tqdm
import timm
import torch.nn as nn
from torchsummary import summary
from torchvision import transforms
import time
from torch.utils.data import DataLoader
import torch.nn.functional as F
from sklearn.metrics import roc_curve, auc


In [5]:
data = np.load('/kaggle/input/gsoclensing/dataset/train/no/5955.npy')

In [6]:
data.shape

(1, 150, 150)

In [7]:
train_data = {}
for i in (range(len(os.listdir(path.maindir)))):
    for j in tqdm(os.listdir(os.path.join(path.maindir,os.listdir(path.maindir)[i]))):
        train_data[os.path.join(os.path.join(path.maindir,os.listdir(path.maindir)[i],j))]=i
        
valid_data = {}
for i in (range(len(os.listdir(path.valid)))):
    for j in tqdm(os.listdir(os.path.join(path.valid,os.listdir(path.valid)[i]))):
        valid_data[os.path.join(os.path.join(path.valid,os.listdir(path.valid)[i],j))]=i


100%|██████████| 2500/2500 [00:00<00:00, 2759.45it/s]


In [8]:
imgtrans = transforms.Compose([
    transforms.ToPILImage(mode='L'),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
]
)

In [9]:
class CustomDataset():
    def __init__(self,dictt,trans):
        self.trans = trans
        self.dic = dictt
    def __len__(self):
        return len(self.dic)
    def __getitem__(self,idx):
        keys = list(self.dic)[idx]
        data = np.load(keys)
        data = torch.tensor(data)
        data = self.trans(data)
        values = list(self.dic.values())[idx]
        return {
            "data":torch.tensor(data,dtype = torch.float32),
            "value":torch.tensor(values)
        }

    
            

In [10]:
custom = CustomDataset(train_data,imgtrans)


In [11]:
len(custom)

30000

In [12]:
custom[0]["data"]

/tmp/ipykernel_33/3178738305.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "data":torch.tensor(data,dtype = torch.float32),


tensor([[[-0.9686, -0.9843, -0.9843,  ..., -0.9765, -0.9922, -0.9843],
         [-0.9765, -0.9843, -0.9686,  ..., -0.9765, -0.9843, -0.9765],
         [-0.9843, -0.9765, -0.9686,  ..., -0.9765, -0.9843, -0.9922],
         ...,
         [-0.9686, -0.9922, -0.9765,  ..., -0.9843, -0.9843, -0.9843],
         [-0.9765, -0.9922, -0.9843,  ..., -0.9922, -0.9765, -0.9686],
         [-0.9686, -0.9686, -0.9686,  ..., -0.9843, -0.9922, -0.9686]]])

In [13]:
def deltamat(shape):
    kk = np.zeros((shape,shape))
    for i in range(shape):
        for j in range(shape):
            kk[i][j] = (((i+1)**2)+((j+1)**2))**0.5
    return torch.tensor(kk,dtype = torch.float32)
positiontensor = deltamat(224)
positiontensor.to("cuda")

tensor([[  1.4142,   2.2361,   3.1623,  ..., 222.0023, 223.0022, 224.0022],
        [  2.2361,   2.8284,   3.6056,  ..., 222.0090, 223.0090, 224.0089],
        [  3.1623,   3.6056,   4.2426,  ..., 222.0203, 223.0202, 224.0201],
        ...,
        [222.0023, 222.0090, 222.0203,  ..., 313.9554, 314.6633, 315.3728],
        [223.0022, 223.0090, 223.0202,  ..., 314.6633, 315.3696, 316.0775],
        [224.0022, 224.0089, 224.0201,  ..., 315.3728, 316.0775, 316.7838]],
       device='cuda:0')

In [14]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel,self).__init__()
        self.resnet = timm.create_model(
            config.model,
            pretrained = True,
            drop_rate = 0.1
        )
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet.fc = nn.Linear(in_features=512, out_features=3, bias=True)
    def forward(self,x):
        x = self.resnet(x)
        return x
        

In [15]:
class SimpleModel(nn.Module):
    def __init__(self,positionmat,batchsize = config.batchsize):
        super(SimpleModel,self).__init__()
        self.positionmat = positionmat
        self.batchsize = batchsize
        self.encoder = timm.create_model(
            config.model1,
            pretrained = True,
            drop_rate = 0.1
        )
        self.encoder.conv1 = Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.coeff = nn.Sequential(
            nn.Flatten(start_dim=1),
            nn.Linear(1000,150*150)
        )
        self.doosramodel = timm.create_model(
            config.model2,
            pretrained = True,
            drop_rate = 0.1
        )
        self.doosramodel.conv1 = Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1000,3)
        )
                
    def forward(self,x):
        k = self.resnet(x)
        k = self.coeff(k)
        #print(k.shape)
        k = k.reshape(self.batchsize,224,224)
        angulardistort = k*self.positionmat
        print(angulardistort.shape)
        sourced_image = x-angulardistort
        features = self.doosramodel(sourced_image)
        prob = self.fc(features)
        return prob
        

'class SimpleModel(nn.Module):\n    def __init__(self,positionmat,batchsize = config.batchsize):\n        super(SimpleModel,self).__init__()\n        self.positionmat = positionmat\n        self.batchsize = batchsize\n        self.resnet = timm.create_model(\n            config.model1,\n            pretrained = True,\n            drop_rate = 0.1\n        )\n        self.coeff = nn.Sequential(\n            nn.Flatten(),\n            nn.Linear(1000,224*224)\n        )\n        self.doosramodel = timm.create_model(\n            config.model2,\n            pretrained = True,\n            drop_rate = 0.1\n        )\n        self.fc = nn.Sequential(\n            nn.Flatten(),\n            nn.Linear(1000,3)\n        )\n                \n    def forward(self,x):\n        k = self.resnet(x)\n        k = self.coeff(k)\n        #print(k.shape)\n        k = k.reshape(self.batchsize,224,224)\n        angulardistort = k*self.positionmat\n        print(angulardistort.shape)\n        sourced_image = x

In [16]:
simple = SimpleModel()
print(simple)
xx = torch.randn(size=(1,1,224,224))

simple(xx)

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

SimpleModel(
  (resnet): ResNet(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1

tensor([[ 0.0025, -0.1034,  0.0846]], grad_fn=<AddmmBackward0>)

In [17]:
simple(custom[0]["data"].unsqueeze(0))

/tmp/ipykernel_33/3178738305.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "data":torch.tensor(data,dtype = torch.float32),


tensor([[0.3846, 0.1204, 0.1341]], grad_fn=<AddmmBackward0>)

In [18]:
traindataset = CustomDataset(train_data,imgtrans)
validdataset  =CustomDataset(valid_data,imgtrans)

trainloader = DataLoader(traindataset,batch_size = 32,shuffle = True)
testloader = DataLoader(validdataset,batch_size = 32,shuffle = True)


In [19]:
from torch.optim.lr_scheduler import OneCycleLR


In [20]:
def trainer(model,trainloader,optimizer,criterion,device):
    model.train()
    iterationloss = 0
    counter = 0
    for data in tqdm(trainloader):
        message = data['data'].to(device)
        target = data['value'].to(device).squeeze()
        optimizer.zero_grad()
        out = model(message)
        loss = criterion(F.log_softmax(out, dim=1), target)
        loss.backward()
        optimizer.step()
        iterationloss+=loss.item()*message.shape[0]
        counter+=message.shape[0]
    return iterationloss/counter

In [21]:
def tester(model, testloader, criterion, device):
    model.eval()
    iteration_loss = 0
    counter = 0
    correct_predictions = 0
    
    y_true = []
    y_scores = []

    for data in tqdm(testloader):
        message = data['data'].to(device)
        target = data['value'].to(device).squeeze()
        
        with torch.no_grad():
            out = model(message)
            loss = criterion(F.log_softmax(out, dim=1), target)
            iteration_loss += loss.item() * message.shape[0]
            
            # Count correct predictions
            predicted_classes = torch.argmax(F.log_softmax(out, dim=1), dim=1)
            correct_predictions += (predicted_classes == target).sum().item()
            """y_true.extend(target.cpu().numpy())
            y_scores.extend(out.cpu().numpy())"""

        counter += message.shape[0]

    accuracy = correct_predictions / counter
    print("Accuracy:", accuracy)
    return iteration_loss / counter


In [22]:
meramodel = SimpleModel()
meramodel.to(device)
optimizer = torch.optim.RAdam(meramodel.parameters(),lr = 0.1)
criterion = nn.CrossEntropyLoss()
scheduler = scheduler = OneCycleLR(
        optimizer,
        max_lr=1e-1,
        epochs=config.EPOCHS,
        steps_per_epoch=938,
        pct_start=0.1,
        anneal_strategy="cos",
        final_div_factor=100)

In [ ]:
trainloss =[]
valloss = []
bestloss = np.inf
for i in range(config.EPOCHS):
    print("Epoch no. =" ,i+1)
    print("-"*80)
    trainl = trainer(meramodel,trainloader,optimizer,criterion,device)
    vall = tester(meramodel,testloader,criterion,device)
    trainloss.append(trainl)
    valloss.append(vall)
    scheduler.step(vall)
    print("TrainLoss = ",trainl," ","ValidationLoss = ",vall)
    lr=scheduler.get_last_lr()[0]
    print("lr = ",lr)
    if bestloss>vall:
        bestloss = vall
        torch.save({
            'model':meramodel.state_dict(),
            'optimizer':optimizer.state_dict()},
        './Bestmodel.model'
        )
        print("Best Model Saved")
    print("="*80)

Epoch no. = 1
--------------------------------------------------------------------------------


  0%|          | 0/938 [00:00<?, ?it/s]/tmp/ipykernel_33/3178738305.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "data":torch.tensor(data,dtype = torch.float32),
 70%|███████   | 657/938 [04:18<01:51,  2.52it/s]

In [ ]:
train_loop()


In [ ]:
class LensingLayer(nn.Module):
    def __init__(self,)

In [ ]:
import torch
x  =torch.tensor([[1,1],[1,1]])
y = torch.tensor([[2,3],[4,6]])

In [ ]:
x*y